In [1]:
import os
import pandas as pd
import swisseph as swe
from swe_const import *
from vedicastro.utils import *
from vedicastro import VedicAstro as va

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
se_version_major, se_version_minor, se_version_patch = swe.version.split('.')
print(se_version_major, se_version_minor, se_version_patch)

2 10 03


In [4]:
swe.get_library_path()
# SE_EPH_PATH = os.getenv('SE_EPHE_PATH')
# print("SWE_EPH_FILE_PATH:",SE_EPH_PATH)
# swe.set_ephe_path(SE_EPH_PATH)

'/opt/miniconda3/envs/astro/lib/python3.11/site-packages/swisseph.cpython-311-darwin.so'

In [5]:
def decimal_to_dms(decimal_deg: float):
    deg, mins, sec, fractional_sec, sign_idx = swe.split_deg(decimal_deg, roundflag = swe.SPLIT_DEG_ZODIACAL )
    sec += fractional_sec  # Add fractional seconds to seconds
    return RASHIS[sign_idx], f"{deg:02d}:{mins:02d}:{int(sec):02d}"

def get_rl_nl_sl_data(deg : float):
    """
    Returns the  Rashi (Sign) Lord, Nakshatra, Nakshatra Pada, Nakshatra Lord, Sub Lord and Sub Sub Lord 
    corresponding to the given degree.
    """
    from vedicastro.VedicAstro import NAKSHATRAS, SIGN_LORDS
    duration = [7, 20, 6, 10, 7, 18, 16, 19, 17]

    lords = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]

    star_lords = lords * 3 ## lords for the 27 Nakshatras

    ## Compute Sign lords
    sign_deg = deg % 360  # Normalize degree to [0, 360)
    sign_index = int(sign_deg // 30)  # Each zodiac sign is 30 degrees
    
    # Compute Nakshatra details
    nakshatra_deg = sign_deg % 13.332  # Each nakshatra is 13.332 degrees
    nakshatra_index = int(sign_deg // 13.332)  # Find the nakshatra index
    pada = int((nakshatra_deg % 13.332) // 3.325) + 1  # Each pada is 3.325 degrees

    # Ensure nakshatra_index is within bounds
    nakshatra_index = nakshatra_index % len(NAKSHATRAS)        

    # Compute SubLords
    deg = deg - 120 * int(deg / 120)
    degcum = 0
    i = 0

    while i < 9:
        deg_nl = 360 / 27
        j = i
        while True:
            deg_sl = deg_nl * duration[j] / 120
            k = j
            while True:
                deg_ss = deg_sl * duration[k] / 120
                degcum += deg_ss
                if degcum >= deg:
                    return {"Nakshatra": NAKSHATRAS[nakshatra_index] + "-" + str(pada), 
                            "RasiLord": SIGN_LORDS[sign_index], "NakshatraLord": star_lords[nakshatra_index],
                            "SubLord": lords[j], "SubSubLord": lords[k] }
                k = (k + 1) % 9
                if k == j:
                    break
            j = (j + 1) % 9
            if j == i:
                break
        i += 1  

### Common `SideReal` Ayanamsas for Vedic Astrology
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/krishnamurti_ayanamshas.html
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/spica_citra_tradition_and_lahiri_ayanamsha.html


In [6]:
SWE_AYANAMSHAS = { "Krishnamurti" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_VP291": swe.SIDM_KRISHNAMURTI_VP291,
                    "Lahiri": swe.SIDM_LAHIRI, "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
                    "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC, 
                }

In [7]:
## Case 1
# year = 2024
# month = 8
# day = 14
# hour = 22
# minute = 31
# secs = 31
# latitude, longitude, utc = 28.6334, 77.2834, "+5:30" ## Delhi

In [8]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [9]:
from pytz import timezone
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

### Results using native `pyswisseph` library


In [10]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_start

In [11]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

results = []  # List to store the results

for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    swe.set_jpl_file("de441.eph")
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags= swe.FLG_SIDEREAL | swe.FLG_JPLEPH )
    
    ## Compute results for only Ascendant
    # asc = cusps[0]
    # sign, sign_dms = decimal_to_dms(asc)
    # res = { 'Ayanamsa': ayan, 'AscSign': sign, 'AscDecLong': asc, 'AscDMS': sign_dms}
    # res.update(get_rl_nl_sl_data(deg = asc))
    # results.append(res)

    ## Compute data for all cusps
    for idx, cusp in zip(range(1, len(cusps)+1),cusps):
        sign, sign_dms = decimal_to_dms(cusp)
        res = {'Ayanamsa': ayan, "HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res.update(get_rl_nl_sl_data(deg = cusp))
        results.append(res)

    
# Convert the list of dictionaries into a pandas DataFrame
df_swisseph = pd.DataFrame(results)

### Results from `VedicAstro` which uses `flatlib`

In [12]:
flatlib_ayanamshas =  ["Krishnamurti", "Krishnamurti_VP291", "Lahiri", "Lahiri_1940", "Lahiri_VP285", "Lahiri_ICRC"]
house_system = "Placidus"

flatlib_results = []
for ayan in flatlib_ayanamshas:
    vhd = va.VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc_offset, 
                            latitude = latitude, longitude = longitude, ayanamsa = ayan, house_system = house_system)
    chart = vhd.generate_chart()

    ## Compute results for only the Ascendant
    # from flatlib import const
    # asc = chart.get(const.ASC)
    # asc_sign, asc_dms = str(asc).split()[1:]
    # res = {"Ayanamsa" : ayan, "AscSign": asc.sign, "AscDecLong": asc.lon, "AscDMS" : asc_dms[:-1]}
    # res.update(get_rl_nl_sl_data(deg = asc.lon))
    # flatlib_results.append(res)

    ## Compute results for all houses
    for house in chart.houses:
        _, house_sign, house_dms, _ = str(house).split()
        res = {"Ayanamsa" : ayan, "HouseID": house.id[5:],"Sign": house_sign, "DecLong": house.lon, "DMS" : house_dms}
        res.update(get_rl_nl_sl_data(deg = house.lon))
        flatlib_results.append(res)
        

In [13]:
df_flatlib = pd.DataFrame(flatlib_results)

In [14]:
# Display them side by side with titles using HTML display with formatted string
def display_side_by_side(df1_html: str, df2_html: str):
    html = f"""
    <b>Housewise Results for :</b> {localized_time} at <b>Lat:</b>{latitude} | <b>Long:</b> {longitude}
    <div style='display:flex; justify-content:space-between;'>
        <div style='width: 45%;'>
            <h2>Swisseph - House Results</h2>
            {df1_html}
        </div>
        <div style='width: 50%;'>
            <h2>Flatlib - House Results</h2>
            {df2_html}
        </div>
    </div>
    """
    display(HTML(html))

In [15]:
## Ayanamsa-wise comparison
req_ayanamsa = "Krishnamurti_VP291"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [16]:
## Ayanamsa-wise comparison
req_ayanamsa = "Krishnamurti"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [17]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [18]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_1940"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [19]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_VP285"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [20]:
## Ayanamsa-wise comparison
req_ayanamsa = "Lahiri_ICRC"
df1_html = df_swisseph[df_swisseph["Ayanamsa"] == req_ayanamsa].to_html()
df2_html = df_flatlib[df_flatlib["Ayanamsa"] == req_ayanamsa].to_html()
display_side_by_side(df1_html, df2_html)

In [21]:
# Convert DataFrames to HTML and concatenate them with some space in between
df1_html = df_swisseph.to_html()
df2_html = df_flatlib.to_html()
display_side_by_side(df1_html, df2_html)